In [2]:
import mne
import numpy as np
import xml.etree.ElementTree as ET
import math
import os
import re
import glob
import pywt

# ===== 参数设置 =====
edf_dir = "/data02/mesa/edfs"
xml_dir = "/data02/mesa/annotations-events-nsrr"
output_base_dir = "/data02/mesa/apnea"
channels = ['EEG1', 'EEG3', 'EOG-L', 'EMG', 'Pres']
n_files = 3000
resample_rate = 100
epoch_length = 30  # seconds
# ====================

def wavelet_denoise(signal, wavelet='sym6', level=5, mode='soft', boundary_mode='symmetric'):
    if signal.ndim != 2:
        raise ValueError("输入数据必须为二维")
    denoised = np.zeros_like(signal)
    for i in range(signal.shape[0]):
        ch = signal[i]
        max_lvl = pywt.dwt_max_level(len(ch), pywt.Wavelet(wavelet))
        actual_lvl = min(level, max_lvl)
        coeffs = pywt.wavedec(ch, wavelet, level=actual_lvl, mode=boundary_mode)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745
        threshold = sigma * np.sqrt(2 * np.log(len(ch)))
        coeffs_thresh = [coeffs[0]] + [pywt.threshold(c, threshold, mode=mode) for c in coeffs[1:]]
        ch_denoised = pywt.waverec(coeffs_thresh, wavelet, mode=boundary_mode)
        denoised[i, :len(ch_denoised)] = ch_denoised[:len(ch)]
    return denoised

def standardize_signal(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    return (signal - mean) / (std if std > 0 else 1)

def preprocess_channel(signal, ch_name, fs):
    if ch_name in ['EEG1', 'EEG3']:
        return standardize_signal(wavelet_denoise(signal[np.newaxis, :])[0])
    elif ch_name == 'EOG-L':
        return standardize_signal(wavelet_denoise(signal[np.newaxis, :], level=3)[0])
    elif ch_name in ['EMG', 'Pres']:
        return signal
    else:
        raise ValueError(f"未知通道: {ch_name}")

def preprocess_all_channels(data, ch_names, fs):
    processed = np.zeros_like(data)
    for i, ch in enumerate(ch_names):
        processed[i] = preprocess_channel(data[i], ch, fs)
    return processed

def generate_apnea_annotations_binary(xml_path, epoch_length=30):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    event_map = {
        'Central apnea': 1,
        'Hypopnea': 1,
        'Obstructive apnea': 1
    }
    apnea_events = []
    max_time = 0
    for event in root.iter('ScoredEvent'):
        concept = event.findtext('EventConcept', '').split('|')[0].strip()
        if concept not in event_map:
            continue
        start = float(event.findtext('Start', '0'))
        duration = float(event.findtext('Duration', '0'))
        end = start + duration
        apnea_events.append({'start': start, 'end': end, 'label': 1})
        max_time = max(max_time, end)

    total_epochs = math.ceil(max_time / epoch_length)
    annotations = [{'epoch': i, 'label': 0} for i in range(total_epochs)]
    for e in apnea_events:
        s_ep = int(e['start'] // epoch_length)
        e_ep = int((e['end'] - 1e-9) // epoch_length)
        for i in range(s_ep, e_ep + 1):
            if 0 <= i < total_epochs:
                annotations[i]['label'] = 1
    return annotations, max_time

def generate_stage_annotations(xml_path, epoch_length=30, total_duration=None):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    events = []
    for ev in root.iter('ScoredEvent'):
        if ev.findtext('EventType') != 'Stages|Stages':
            continue
        concept = ev.findtext('EventConcept', '')
        parts = concept.split('|')
        if len(parts) < 2 or not parts[1].isdigit():
            continue
        label = int(parts[1])
        label = 4 if label == 5 else label
        try:
            start = float(ev.findtext('Start'))
            duration = float(ev.findtext('Duration'))
        except:
            continue
        if duration <= 0:
            continue
        events.append({'start': start, 'end': start + duration, 'label': label})
    max_time = total_duration or max([e['end'] for e in events] + [0])
    total_epochs = math.ceil(max_time / epoch_length)
    annotations = [{'epoch': i, 'label': 0} for i in range(total_epochs)]
    for e in events:
        s = int(e['start'] // epoch_length)
        e_ = int((e['end'] - 1e-9) // epoch_length)
        for i in range(s, min(e_ + 1, total_epochs)):
            annotations[i]['label'] = e['label']
    return annotations

def process_apnea_epoch_segments(edf_path, xml_path, output_dir):
    apnea_anns, apnea_max_time = generate_apnea_annotations_binary(xml_path, epoch_length)
    stage_anns = generate_stage_annotations(xml_path, epoch_length, apnea_max_time)

    raw = mne.io.read_raw(edf_path, preload=True, verbose=False)
    if resample_rate:
        raw.resample(resample_rate)

    raw.pick_channels([ch for ch in channels if ch in raw.ch_names])
    fs = raw.info['sfreq']
    duration = raw.n_times / fs
    max_time = min(duration, apnea_max_time)
    aligned_time = (max_time // epoch_length) * epoch_length
    raw.crop(tmax=aligned_time)

    data = raw.get_data()
    ch_names = raw.ch_names
    data = preprocess_all_channels(data, ch_names, fs)

    total_epochs = int(aligned_time // epoch_length)
    samples_per_epoch = int(epoch_length * fs)
    apnea_anns = apnea_anns[:total_epochs]
    stage_anns = stage_anns[:total_epochs]

    os.makedirs(output_dir, exist_ok=True)
    saved = 0
    for i in range(total_epochs):
        if stage_anns[i]['label'] == 0:
            continue  # skip wake
        seg = data[:, i * samples_per_epoch: (i + 1) * samples_per_epoch]
        label = apnea_anns[i]['label']
        fname = os.path.join(output_dir, f"ep{i:04d}_label{label}.npy")
        np.save(fname, seg)
        saved += 1

    print(f"[INFO] {os.path.basename(edf_path)}: 共保存 {saved} 个非清醒期片段")

# ====== 主程序入口 ======
if __name__ == "__main__":
    edf_files = glob.glob(os.path.join(edf_dir, "mesa-sleep-*.edf"))
    edf_files.sort(key=lambda x: int(re.search(r"mesa-sleep-(\d+)\.edf", x).group(1)))

    if isinstance(n_files, int):
        edf_files = edf_files[1250:n_files]

    print(f"[INFO] 处理 {len(edf_files)} 个文件")
    for idx, edf_path in enumerate(edf_files, 1):
        try:
            file_id = re.search(r"mesa-sleep-(\d+)\.edf", os.path.basename(edf_path)).group(1)
            xml_path = os.path.join(xml_dir, f"mesa-sleep-{file_id}-nsrr.xml")
            output_dir = os.path.join(output_base_dir, file_id)
            if not os.path.exists(xml_path):
                print(f"[SKIP] 缺失标注文件: {xml_path}")
                continue
            print(f"\n[{idx}/{len(edf_files)}] 正在处理: {file_id}")
            process_apnea_epoch_segments(edf_path, xml_path, output_dir)
        except Exception as e:
            print(f"[ERROR] 文件 {edf_path} 处理失败: {e}")


[INFO] 处理 806 个文件

[1/806] 正在处理: 4219
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1467667/1466765677.py:126: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels([ch for ch in channels if ch in raw.ch_names])


[INFO] mesa-sleep-4219.edf: 共保存 896 个非清醒期片段

[2/806] 正在处理: 4223
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1467667/1466765677.py:126: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels([ch for ch in channels if ch in raw.ch_names])


[INFO] mesa-sleep-4223.edf: 共保存 781 个非清醒期片段

[3/806] 正在处理: 4224
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

[INFO] mesa-sleep-6810.edf: 共保存 672 个非清醒期片段

[805/806] 正在处理: 6811
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1467667/1466765677.py:126: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels([ch for ch in channels if ch in raw.ch_names])


[INFO] mesa-sleep-6811.edf: 共保存 727 个非清醒期片段

[806/806] 正在处理: 6812
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1467667/1466765677.py:126: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels([ch for ch in channels if ch in raw.ch_names])


[INFO] mesa-sleep-6812.edf: 共保存 569 个非清醒期片段
